# Import

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Config

In [ ]:
SIM_FUTURE_MONTH = 12*2
SIM_NUM = 1000
COEF_ESTIMATE_MONTH = 12*5
EPSILON_ESTIMATE_MONTH = 12*2
PATH_INPUT = "/Users/makoto/Desktop/crude-oil.xls"

# Common Function

In [ ]:
def add_suffix_to_level0(df, suffix, level=0):
    df.columns = pd.MultiIndex.from_tuples(
        [
            tuple(f"{val}{suffix}" if i == level else val for i, val in enumerate(col))
            for col in df.columns
        ]
    )
    return df

In [ ]:
def use_prev_row(func):
    prev_row = {}
    def wrapper(cur_row, **kwargs):
        val = func(cur_row, prev_row)
        prev_row.update(val)
        return val
    def set_prev_row(row):
        nonlocal prev_row
        prev_row = row
    wrapper.set_prev_row = set_prev_row
    return wrapper


# MODEL CONSTRUCTION

In [ ]:
#データ読み込み
org = pd.read_excel(
    PATH_INPUT,
    skiprows=5,
    usecols=[2,3,4],
)
org.columns=pd.MultiIndex.from_product([["org"], org.columns])
org = org.iloc[400:].reset_index(drop=True)

In [ ]:
#中間ファイル生成
org_diff = add_suffix_to_level0(org.diff(), "_diff")
org_diff_shift = add_suffix_to_level0(org_diff.shift(), "_shift")
org = pd.concat([org, org_diff, org_diff_shift], axis=1)

In [ ]:
#モデル構築
model = sm.tsa.VAR(org["org_diff"].iloc[-COEF_ESTIMATE_MONTH:].dropna())
results = model.fit(1)

In [ ]:
#in-sample予測
def predict(x, coef=results.params.values):
    var = np.insert(x.values, 0, 1)
    insample_pred = np.dot(var, coef)
    return pd.Series(insample_pred, index=x.keys())
past_in_pred = org[["org_diff_shift"]].apply(predict, axis=1)
past_in_pred = add_suffix_to_level0(past_in_pred, "_to_pred")
past = pd.concat([org, past_in_pred], axis=1)

In [ ]:
#残差計算
past_resid = add_suffix_to_level0(past[["org_diff"]] - past["org_diff_shift_to_pred"].values, "_resid")
past = pd.concat([past, past_resid], axis=1)

In [ ]:
#撹乱項の分散共分散行列の推定
cov = past["org_diff_resid"].iloc[-EPSILON_ESTIMATE_MONTH:].cov()

In [ ]:
#sim用のデータフレーム作成
def make_sim_df(df, cov=cov):
    columns = pd.MultiIndex.from_tuples([("epsilon", k2) for (k1, k2) in df.columns if k1=="org"])
    return pd.DataFrame(
        np.random.multivariate_normal(mean=[0,0,0], cov=cov.values, size=SIM_FUTURE_MONTH),
        columns=columns,
        index=range(len(df),len(df)+SIM_FUTURE_MONTH),
    )

In [ ]:
#差分の将来予測
@use_prev_row
def predict_future_org_diff(cur_row, prev_row, coef=results.params.values):
    var = np.array(list(prev_row.values))
    var = np.insert(var, 0, 1)
    result = np.dot(var, coef) + cur_row.values
    return pd.Series(result, index = prev_row.keys())

In [ ]:
#原系列の将来予測
@use_prev_row
def predict_future_org(cur_row, prev_row):
    return pd.Series(cur_row.values + prev_row.values, index= prev_row.keys())

In [ ]:
#本質的な実行main
def simulate(past):
    sim = make_sim_df(past)
    predict_future_org_diff.set_prev_row(past[["org_diff"]].iloc[-1])
    future_org_diff = sim[["epsilon"]].apply(predict_future_org_diff, axis=1)
    sim = pd.concat([sim, future_org_diff],axis=1)
    predict_future_org.set_prev_row(past[["org"]].iloc[-1])
    future_org = sim[["org_diff"]].apply(predict_future_org, axis=1)
    sim = pd.concat([sim, future_org], axis=1)
    return pd.concat([past, sim])

In [ ]:
def make_analysis_data(past):
    return pd.concat([simulate(past) for n in range(SIM_NUM)], axis=1)


In [ ]:
output = make_analysis_data(past)

# APPENDIX

In [ ]:
plot_acf(past.dropna()[("org_diff", "ブレント")])
plot_pacf(past.dropna()[("org_diff", "ブレント")])
plt.show()

In [ ]:
results.summary()

In [ ]:
output[("org", "ブレント")].plot(legend=False, linewidth=.5, figsize=(10,3))

In [ ]:
slice_data = output[("org", "ブレント")].iloc[-1,:]
ax = slice_data.hist(bins=100, figsize=(10,3))
ax.axvline(slice_data.quantile(.95), color='r')
ax.axvline(slice_data.quantile(.05), color='r')

In [ ]:
sigma = np.diagonal(cov**(1/2))
coef = 1.96
lower = past["org_diff_shift_to_pred"] - coef*sigma
upper = past["org_diff_shift_to_pred"] + coef*sigma

In [ ]:
fig = plt.figure(figsize=(10,3))
ax = fig.add_subplot()
past["org_diff"]["ブレント"].plot(ax=ax)
kwargs = {"ax":ax, "linestyle":"dotted", "color":"r"}
lower["ブレント"].plot(**kwargs)
upper["ブレント"].plot(**kwargs)